## Redes Neurais Artificiais 2020.1

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
**Aluno(a):** Erik Atilio Silva Rey  
**Aluno(a):** Enrique Leão Barbosa Izel  


Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

In [3]:
## Reservado para a importação de bibliotecas
import numpy as np
import pandas as pd
import random
import math
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão

In [7]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/projeto-pratico-3-equipe-ultra-mega-dinamite/notebooks/data/covtype.csv')

2. Elimine todas as colunas relativas aos atributos categóricos

In [8]:
# Função para remover colunas dos atributos categóricos
def remove_columns(column, n):
    columns_to_remove = []
    for i in range(1,n+1):
        columns_to_remove.append(column+str(i))
    return df.drop(columns_to_remove, axis=1)

In [9]:
df = remove_columns('Soil_Type', 40)
df = remove_columns('Wilderness_Area', 4)

3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X

In [11]:
df_y = df['Cover_Type']
df_x = df.loc[:, 'Elevation':'Horizontal_Distance_To_Fire_Points']

4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória

In [12]:
# Particao holdout para treino (70%) e teste (30%)
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, train_size=0.7, random_state=42)

5. Efetue o escalonamento dos atributos

In [13]:
# Escalonando os atributos
x_train_std = (x_train - np.mean(x_train))/np.std(x_train)
x_test_std = (x_test - np.mean(x_train))/np.std(x_train)

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.

### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  

In [10]:
# Criando e treinando a rede neural multilayer perceptron
nmp = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="adam", 
                    random_state=1, max_iter=300, 
                    verbose=True).fit(x_train_std, y_train)

Iteration 1, loss = 0.92677146
Iteration 2, loss = 0.70663965
Iteration 3, loss = 0.68742275
Iteration 4, loss = 0.67804785
Iteration 5, loss = 0.67173330
Iteration 6, loss = 0.66795662
Iteration 7, loss = 0.66536055
Iteration 8, loss = 0.66363719
Iteration 9, loss = 0.66249517
Iteration 10, loss = 0.66159913
Iteration 11, loss = 0.66098261
Iteration 12, loss = 0.66031835
Iteration 13, loss = 0.65972541
Iteration 14, loss = 0.65897054
Iteration 15, loss = 0.65844078
Iteration 16, loss = 0.65794027
Iteration 17, loss = 0.65739333
Iteration 18, loss = 0.65696161
Iteration 19, loss = 0.65668155
Iteration 20, loss = 0.65645934
Iteration 21, loss = 0.65621898
Iteration 22, loss = 0.65608783
Iteration 23, loss = 0.65586838
Iteration 24, loss = 0.65560923
Iteration 25, loss = 0.65548498
Iteration 26, loss = 0.65533209
Iteration 27, loss = 0.65523662
Iteration 28, loss = 0.65506551
Iteration 29, loss = 0.65495128
Iteration 30, loss = 0.65482173
Iteration 31, loss = 0.65462318
Iteration 32, los

6. Com o modelo em questão, após o treinamento, apresente:

In [32]:
# Respostas para o conjunto de teste
y_pred = nmp.predict(x_test_std)

# Matriz de confusao
muddle_matrix = confusion_matrix(y_test, y_pred)

In [34]:
# Função que imprime a matriz de confusão
def print_muddle_matrix(muddle_matrix):
    data = np.array(muddle_matrix)
    index, columns = [],[]
    for i in range(1,len(data)+1):
        index.append(i)
        columns.append(i)
    return pd.DataFrame(data = data, index = index, columns = columns)

6.1 Matriz de confusão para o conjunto de teste  

In [52]:
print_muddle_matrix(muddle_matrix)

,1,2,3,4,5,6,7
1,46387,17012,4,0,0,0,40
2,17921,65840,1185,0,0,0,1
3,0,2444,8285,0,0,0,0
4,0,5,830,0,0,0,0
5,0,2878,16,0,0,0,0
6,0,1854,3374,0,0,0,0
7,5976,23,0,0,0,0,229


6.2 Acurácia  

In [14]:
round(accuracy_score(y_test, y_pred), 4)

0.7148

6.3 F-Score  

In [15]:
round(f1_score(y_test, y_pred, average='macro'), 4)

0.5288

6.4 Precisão  

In [16]:
round(precision_score(y_test, y_pred, average='macro'), 4)

0.6527

6.5 Revocação  

In [17]:
round(recall_score(y_test, y_pred, average='macro'), 4)

0.4919

7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  

In [ ]:
f1_scores, accuracy = np.zeros(10), np.zeros(10)

nmp = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="adam", 
                    random_state=1, max_iter=300, 
                    verbose=False)

In [ ]:
for i in range (10):
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, train_size=0.7, random_state=1)
    x_train_std = (x_train - np.mean(x_train))/np.std(x_train)
    x_test_std = (x_test - np.mean(x_train))/np.std(x_train)
    nmp.fit(x_train_std, y_train)
    y_pred = nmp.predict(x_test_std)
    
    f1_scores[i] = f1_score(y_test, y_pred, average='macro')
    accuracy[i] = accuracy_score(y_test, y_pred)

7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  

In [ ]:
print(round(f1_scores.mean(),5), round(accuracy.mean(),5), round(f1_scores.std(),5), round(accuracy.std(),5))

0.514 0.71611 0.0 0.0


8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  

In [ ]:
f1_scores, accuracy = np.zeros(10), np.zeros(10)

nmp = MLPClassifier(hidden_layer_sizes=(10),activation="relu", solver="sgd", 
                    random_state=1, max_iter=300, 
                    verbose=False)

In [ ]:
for i in range (10):
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, train_size=0.7, random_state=1)
    x_train_std = (x_train - np.mean(x_train))/np.std(x_train)
    x_test_std = (x_test - np.mean(x_train))/np.std(x_train)
    nmp.fit(x_train_std, y_train)
    y_pred = nmp.predict(x_test_std)
    
    f1_scores[i] = f1_score(y_test, y_pred, average='macro')
    accuracy[i] = accuracy_score(y_test, y_pred)

8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  

In [ ]:
print(round(f1_scores.mean(),5), round(accuracy.mean(),5), round(f1_scores.std(),5), round(accuracy.std(),5))

0.47272 0.71761 0.0 0.0


9. Houve influência da escolha do otimizador no desempenho da rede?  
  **R=** Sim, a média e desvio padrão da Acurácia e do F-Score com os hiperparâmetros solver 'adam' e 'sgd'. O modelo com o solver **'adam' se saiu levemente melhor**, pois teve uma maior pontuação para média de f-score e para média de acurácia também.

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha

In [18]:
activation_function, hyper_parameter, times, layers, config = ["identity", "logistic", "relu", "tanh"], ["adam", "sgd"], [100, 150, 200], [1, 2], []

for i in range(10):
  type_time = random.randrange(1, 1000000, 1)%3
  type_hyper_parameter, quantity_layers = random.randrange(1, 1000000, 1)%2, random.randrange(1, 1000000, 1)%2
  type_activation_function = random.randrange(1, 1000000, 1)%4
  quantity_neurons = random.randrange(1, 20, 1)
  aux = random.randrange(1, quantity_neurons, 1)
  disposition_neurons = quantity_neurons if layers[quantity_layers]== 1 else (aux, quantity_neurons - aux)
  config.append({
      "Qtd Camadas": layers[quantity_layers],
      "Função de Ativação": activation_function[type_activation_function],
      "Hiperparâmetro": hyper_parameter[type_hyper_parameter],
      "Qtd Epocas": times[type_time],
      "Qtd Nêuronios": quantity_neurons,
      "Disposição de Nêuronios": disposition_neurons
      })

In [29]:
result = pd.DataFrame(config)
result.index += 1
result

,Qtd Camadas,Função de Ativação,Hiperparâmetro,Qtd Epocas,Qtd Nêuronios,Disposição de Nêuronios
1,2,tanh,adam,150,11,"(3, 8)"
2,1,relu,sgd,150,8,8
3,1,logistic,sgd,100,13,13
4,1,tanh,adam,200,6,6
5,2,relu,sgd,150,15,"(1, 14)"
6,2,identity,adam,100,2,"(1, 1)"
7,1,relu,adam,150,6,6
8,2,logistic,adam,200,6,"(5, 1)"
9,2,tanh,adam,100,9,"(7, 2)"
10,1,tanh,sgd,150,2,2


2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes

In [ ]:
f1_scores, accuracy, average_accuracy, average_f1score, dp_accuracy, dp_f1score = np.zeros(10), np.zeros(10), np.zeros(10), np.zeros(10), np.zeros(10), np.zeros(10)
result.index -= 1

for i in range(0,10):
  nmp = MLPClassifier(hidden_layer_sizes=result["Qtd Camadas"][i],
                      activation=result["Função de Ativação"][i],
                      solver=result["Hiperparâmetro"][i],
                      max_iter=result["Qtd Epocas"][i], verbose=False)
  for j in range(10):
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, train_size=0.7)
    x_train_std = (x_train - np.mean(x_train))/np.std(x_train)
    x_test_std = (x_test - np.mean(x_train))/np.std(x_train)
    nmp.fit(x_train_std, y_train)
    y_pred = nmp.predict(x_test_std)
    
    f1_scores[j] = f1_score(y_test, y_pred, average='macro')
    accuracy[j] = accuracy_score(y_test, y_pred)
  print(f'...{i+1*10}%')
  average_accuracy[i] = round(accuracy.mean(), 5)
  average_f1score[i] = round(f1_scores.mean(), 5)
print(f'{i+1*10}%')

3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  

In [30]:
result["Média Acurácia"] = average_accuracy
result["Média F-Score"] = average_f1score
result.sort_values(["Média Acurácia"], ascending=False)

,Qtd Camadas,Função de Ativação,Hiperparâmetro,Qtd Epocas,Qtd Nêuronios,Disposição de Nêuronios,Média Acurácia,Média F-Score
5,2,relu,sgd,150,15,"(1, 14)",0.69672,0.36240
9,2,tanh,adam,100,9,"(7, 2)",0.69561,0.35513
7,1,relu,adam,150,6,6,0.69552,0.34999
1,2,tanh,adam,150,11,"(3, 8)",0.69502,0.35407
8,2,logistic,adam,200,6,"(5, 1)",0.69478,0.35210
4,1,tanh,adam,200,6,6,0.69460,0.34727
2,1,relu,sgd,150,8,8,0.69445,0.32642
6,2,identity,adam,100,2,"(1, 1)",0.69418,0.37057
10,1,tanh,sgd,150,2,2,0.69390,0.31374
3,1,logistic,sgd,100,13,13,0.69261,0.30342


In [31]:
result.sort_values(["Média F-Score"], ascending=False)

,Qtd Camadas,Função de Ativação,Hiperparâmetro,Qtd Epocas,Qtd Nêuronios,Disposição de Nêuronios,Média Acurácia,Média F-Score
6,2,identity,adam,100,2,"(1, 1)",0.69418,0.37057
5,2,relu,sgd,150,15,"(1, 14)",0.69672,0.36240
9,2,tanh,adam,100,9,"(7, 2)",0.69561,0.35513
1,2,tanh,adam,150,11,"(3, 8)",0.69502,0.35407
8,2,logistic,adam,200,6,"(5, 1)",0.69478,0.35210
7,1,relu,adam,150,6,6,0.69552,0.34999
4,1,tanh,adam,200,6,6,0.69460,0.34727
2,1,relu,sgd,150,8,8,0.69445,0.32642
10,1,tanh,sgd,150,2,2,0.69390,0.31374
3,1,logistic,sgd,100,13,13,0.69261,0.30342


**R=** Segundo a média de Acurácia as melhores foram: 5, 9 e 7. E segundo a média de F-Score as melhores foram: 6, 5 e 9. Para a escolha do top 3 decidimos por na seguinte ordem 6, 5 e 9 pois a acurácia pode ser usada quando se há classes de distribuição semelhante, já no caso da F-Score quando as mesmas estão desbalanceada, no caso deste exemplo.

3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

In [44]:
def fit(layers, function, hiperparam, times):
    nmp = MLPClassifier(hidden_layer_sizes=layers, activation=function, solver= hiperparam, max_iter=times, verbose= False)
    nmp.fit(x_train_std, y_train)
    return confusion_matrix(y_test, nmp.predict(x_test_std))

In [48]:
print_muddle_matrix(fit(result['Qtd Camadas'][6], result['Função de Ativação'][6],result['Hiperparâmetro'][6],result['Qtd Epocas'][6]))

,1,2,3,4,5,6,7
1,43226,18946,2,0,0,0,1269
2,16017,67579,1172,0,0,135,44
3,0,2281,7955,6,0,487,0
4,0,5,821,1,0,8,0
5,0,2878,16,0,0,0,0
6,0,1831,3031,0,0,366,0
7,4519,87,0,0,0,0,1622


In [49]:
print_muddle_matrix(fit(result['Qtd Camadas'][5], result['Função de Ativação'][5],result['Hiperparâmetro'][5],result['Qtd Epocas'][5]))

,1,2,3,4,5,6,7
1,43498,19056,0,0,0,0,889
2,16071,67851,1016,0,0,0,9
3,0,2625,8104,0,0,0,0
4,0,10,825,0,0,0,0
5,0,2893,1,0,0,0,0
6,0,1867,3361,0,0,0,0
7,4839,23,0,0,0,0,1366


In [50]:
print_muddle_matrix(fit(result['Qtd Camadas'][9], result['Função de Ativação'][9],result['Hiperparâmetro'][9],result['Qtd Epocas'][9]))

,1,2,3,4,5,6,7
1,44034,18291,18,0,0,0,1100
2,16909,66874,1156,0,0,0,8
3,0,2331,8398,0,0,0,0
4,0,10,825,0,0,0,0
5,0,2876,18,0,0,0,0
6,0,1921,3307,0,0,0,0
7,4702,23,0,0,0,0,1503


## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_h$.  
  **R=** Segundo a documentação da biblioteca sklearn.neural_network.MLPClassifier informa, os valores para Ni, No são a quantidade de atributos preditores e quantidade de classes. Sendo assim, ambos correspondem a 10 e 7.

2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.

In [61]:
Ni, No, alpha = 10, 7, [0.5, 2, 3]
Nh = [] 

for i in alpha:
  Nh.append(int(i*math.sqrt(Ni*No)))


3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  

In [4]:
activation_function, hyper_parameter, times, layers, config = ["identity", "logistic", "relu", "tanh"], ["adam", "sgd"], [100, 150, 200], [1, 2], []

for i in range(30):
  type_time = random.randrange(1, 1000000, 1)%3
  type_hyper_parameter, quantity_layers = random.randrange(1, 1000000, 1)%2, random.randrange(1, 1000000, 1)%2
  type_activation_function = random.randrange(1, 1000000, 1)%4
  quantity_neurons = random.randrange(1, 20, 1)
  aux = random.randrange(1, quantity_neurons, 1)
  disposition_neurons = quantity_neurons if layers[quantity_layers]== 1 else (aux, quantity_neurons - aux)
  config.append({
      "Qtd Camadas": layers[quantity_layers],
      "Função de Ativação": activation_function[type_activation_function],
      "Hiperparâmetro": hyper_parameter[type_hyper_parameter],
      "Qtd Epocas": times[type_time],
      "Qtd Nêuronios": quantity_neurons,
      "Disposição de Nêuronios": disposition_neurons
      })

In [5]:
result = pd.DataFrame(config)
result.index += 1
result

,Qtd Camadas,Função de Ativação,Hiperparâmetro,Qtd Epocas,Qtd Nêuronios,Disposição de Nêuronios
1,2,tanh,sgd,100,7,"(5, 2)"
2,1,relu,sgd,200,17,17
3,1,relu,adam,200,16,16
4,1,identity,sgd,100,16,16
5,2,logistic,sgd,200,9,"(5, 4)"
6,1,logistic,sgd,200,14,14
7,1,logistic,sgd,100,4,4
8,1,relu,adam,100,4,4
9,1,relu,adam,200,4,4
10,1,logistic,adam,100,19,19


3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia

In [ ]:
f1_scores, accuracy, average_accuracy, average_f1score, dp_accuracy, dp_f1score = np.zeros(30), np.zeros(30), np.zeros(30), np.zeros(30), np.zeros(30), np.zeros(30)
result.index -= 1

for i in range(0,30):
  nmp = MLPClassifier(hidden_layer_sizes=result["Qtd Camadas"][i],
                      activation=result["Função de Ativação"][i],
                      solver=result["Hiperparâmetro"][i],
                      max_iter=result["Qtd Epocas"][i], verbose=False)
  for j in range(30):
    x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, train_size=0.7)
    x_train_std = (x_train - np.mean(x_train))/np.std(x_train)
    x_test_std = (x_test - np.mean(x_train))/np.std(x_train)
    nmp.fit(x_train_std, y_train)
    y_pred = nmp.predict(x_test_std)
    
    f1_scores[j] = f1_score(y_test, y_pred, average='macro')
    accuracy[j] = accuracy_score(y_test, y_pred)
  average_accuracy[i] = round(accuracy.mean(), 5)
  average_f1score[i] = round(f1_scores.mean(), 5)

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

In [ ]:
;(

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?

In [ ]:
;(